In [1]:
import cv2
import pytesseract
import numpy as np
import os
from matplotlib import pyplot as plt

In [2]:
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)

    height, width  = im_data.shape[:2]

    figsize = width / float(dpi), height / float(dpi)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')

    ax.imshow(im_data, cmap='gray')
    plt.show()

In [3]:
imgq=cv2.imread('invoice1.jpg')

In [4]:
h,w,c=imgq.shape

In [5]:
orb=cv2.ORB_create(1000)
kp1,des1 = orb.detectAndCompute(imgq,None)

In [6]:
path='invoice'
mypic = os.listdir(path)
print(mypic)

['invoice2.jpg', 'invoice3.jpg']


In [7]:
for j,y in enumerate(mypic):
    img=cv2.imread(path+ "/" +y)

In [8]:
kp2,des2 = orb.detectAndCompute(img,None)

In [9]:
per=25

In [10]:
bf=cv2.BFMatcher(cv2.NORM_HAMMING)
matches = list(bf.match(des2, des1))
matches.sort(key = lambda x:x.distance)
good=matches[:int(len(matches))]


In [11]:
imgmatch=cv2.drawMatches(img,kp2,imgq,kp1,good[:20],None, flags=2)

In [12]:
srcpoint = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
dstpoint = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1,1,2)

In [13]:
M, _ =cv2.findHomography(srcpoint, dstpoint, cv2.RANSAC, 5.0)

In [14]:
imgscan =cv2.warpPerspective(img,M,(w,h))

In [24]:
roi = [[(1129,252), (1200,274), 'text' , 'invoice'],
[(1080,284), (1201,308), 'text' , 'date'],
[(1081,317), (1200,341), 'text' , 'due date'],
[(128,440), (254,467), 'text' , 'item'],
[(734,439), (780,465), 'text' , 'quantity'],
[(880,439), (992,470), 'text' , 'price'],
[(130,486), (211,513), 'text' , 'item'],
[(732,486), (783,510), 'text' , 'quantity'],
[(897,487), (991,512), 'text' , 'price'],
[(1099,535), (1209,565), 'text' , 'total']]

In [25]:
imgshow = imgscan.copy()
imgmask = np.zeros_like(imgshow)

In [26]:
mydata=[]

In [29]:
for x,r in enumerate(roi):
    cv2.rectangle(imgmask, (r[0][0],r[0][1]), (r[1][0],r[1][1]), (0,255,0), cv2.FILLED)
    imgshow = cv2.addWeighted(imgshow, 0.99, imgmask, 0.1, 0)
    imgcrop = imgscan[r[0][1] : r[1][1], r[0][0] : r[1][0]]
    if r[2] == 'text':
        mydata.append(pytesseract.image_to_string(imgcrop))
   


In [30]:
with open('data.csv', 'a+') as f:
    for data in mydata:
        f.write((str(data)+','))
    f.write('\n')